In [1]:
import matplotlib.pyplot as plt
import matplotlib.path as mplPath
import numpy as np
import math
from valueMaps import ValueMap
from func import readGridDataFromFile

In [2]:
def scaleDownArray(array):
    offsets = [-1,0,1]
    
    smallArray = np.empty((round(array.shape[0]/3), round(array.shape[1]/3)))
    largeArray = array.copy()
    for rowIndex in range(math.floor(largeArray.shape[0] / 3)):
        for columnIndex in range(math.floor(largeArray.shape[1]/3)):
            val = 0
            for i in offsets:
                for j in offsets:
                    val = val + largeArray[(rowIndex)*3+i+1][(columnIndex)*3+j+1]
            smallArray[rowIndex][columnIndex] = val
    return smallArray

In [3]:
# Make maptlotlib show graphs in a new window
%matplotlib qt 

In [29]:
dataDate = "10/07/2023" # Date of the data to be collected in format %D/%M/%Y
dataType = "density" # "irradiance" OR "temperature" OR "density" OR "pressure" OR "rainfall"
gridData = []
units = {"temperature": "Celcius", "irradiance": "MJ/m^2", "pressure": "hPa", "rainfall": "mm", "density": "count"}

In [30]:
if dataType != "density":
    path = f"../main/data/{dataType}/{dataDate.replace('/', '.')}.grid"
else: 
    path = f"../main/data/density/normalizedDensity.grid"
gridData = readGridDataFromFile(path)


In [31]:
scaleDownGrid = False
count = 2

if scaleDownGrid:
    output = mapData.copy()
    for i in range(count):
        processingArray = output.copy()
        processingArray = processingArray[processingArray.shape[0]%3:]
        if(processingArray.shape[1]%3==1):
            np.delete(processingArray, [0], axis=1)
        if(processingArray.shape[1]%2==2):
            np.delete(processingArray, [0,1], axis=1)
        output = scaleDownArray(processingArray)

In [32]:
gridVals = ValueMap(
    gridData["gridSize"], 
    gridData["southWestPos"],
    gridData["cellSize"], 
    gridData["noDataVal"], 
    units[dataType])

southWestPos = gridData["southWestPos"]
gridSize = gridData["gridSize"]
cellSize = gridData["cellSize"]

gridVals.setGridData(gridData["mapData"])

In [34]:
gridVals.gridData.shape

(455, 822)